In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('data/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.shape

(20800, 5)

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape

(18285, 5)

In [11]:
x=df['title']
y=df['label']

In [12]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [13]:
import tensorflow as tf

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [17]:
voc_size=5000

In [22]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [25]:
def preprocess_text(x):
    corpus=[]
    for sent in x:
        review=re.sub('[^a-zA-Z]',' ', sent)
        review=review.lower()
        review=review.split()
        
        review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
        review=' '.join(review)
        
        corpus.append(review)
        
    return corpus
        

In [27]:
processed_text=preprocess_text(x)

In [28]:
processed_text[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [29]:
onehot_text= [one_hot(words, voc_size) for words in processed_text]
onehot_text[:5]

[[1494, 404, 857, 116, 290, 4635, 625, 239, 2951, 2102],
 [2641, 3382, 3199, 600, 2901, 3270, 3772],
 [3851, 2754, 334, 1746],
 [4335, 4058, 2023, 16, 224, 2755],
 [3229, 2901, 2206, 4752, 2811, 247, 2901, 4543, 2773, 215]]

In [30]:
padded_text=pad_sequences(onehot_text, padding='pre', maxlen=20)
padded_text[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1494,
         404,  857,  116,  290, 4635,  625,  239, 2951, 2102],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2641, 3382, 3199,  600, 2901, 3270, 3772],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3851, 2754,  334, 1746],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4335, 4058, 2023,   16,  224, 2755],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3229,
        2901, 2206, 4752, 2811,  247, 2901, 4543, 2773,  215]],
      dtype=int32)

In [31]:
model=Sequential()
model.add(Embedding(voc_size,40, input_length=20))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
import numpy as np
x_train= np.array(padded_text)
y_train= np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_val, Y_train, Y_val= train_test_split(x_train,y_train, test_size=0.3, random_state=10, stratify=y_train)

In [35]:
model.fit(X_train,Y_train, validation_data=(X_val,Y_val),epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 10s 30ms/step - loss: 0.3161 - accuracy: 0.8533 - val_loss: 0.1900 - val_accuracy: 0.9247
Epoch 2/10
200/200 [==============================] - 5s 27ms/step - loss: 0.1410 - accuracy: 0.9440 - val_loss: 0.1811 - val_accuracy: 0.9247
Epoch 3/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0987 - accuracy: 0.9645 - val_loss: 0.2156 - val_accuracy: 0.9249
Epoch 4/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0669 - accuracy: 0.9763 - val_loss: 0.2441 - val_accuracy: 0.9178
Epoch 5/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0447 - accuracy: 0.9848 - val_loss: 0.2814 - val_accuracy: 0.9196
Epoch 6/10
200/200 [==============================] - 8s 38ms/step - loss: 0.0270 - accuracy: 0.9915 - val_loss: 0.3308 - val_accuracy: 0.9187
Epoch 7/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.4367 - val_accuracy: 0.916

In [36]:
y_pred= model.predict(X_val)

172/172 [==============================] - 2s 5ms/step


In [40]:
y_pred= [round(pred[0]) for pred in y_pred]
y_pred[:5]

[1, 1, 0, 1, 1]

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [42]:
confusion_matrix(y_pred,Y_val)

array([[2887,  234],
       [ 222, 2143]])

In [43]:
accuracy_score(y_pred,Y_val)

0.9168793292016041

In [44]:
print(classification_report(y_pred, Y_val))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      3121
           1       0.90      0.91      0.90      2365

    accuracy                           0.92      5486
   macro avg       0.92      0.92      0.92      5486
weighted avg       0.92      0.92      0.92      5486


In [47]:
df_test= pd.read_csv('data/test.csv')

In [49]:
df_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [50]:
df_test.shape

(5200, 4)

In [52]:
df_test.dropna(inplace=True)

In [53]:
df_test.shape

(4575, 4)

In [54]:
processed_text_test=preprocess_text(df_test['title'])
processed_text_test[:5]

['specter trump loosen tongu purs string silicon valley new york time',
 'nodapl nativ american leader vow stay winter file lawsuit polic',
 'tim tebow attempt anoth comeback time basebal new york time',
 'keiser report meme war e',
 'pelosi call fbi investig find russian donald trump breitbart']

In [55]:
onehot_text_test=[one_hot(words, voc_size) for words in processed_text_test]
onehot_text_test[:5]

[[2746, 1881, 550, 1005, 2302, 2813, 3276, 2973, 3329, 1277, 3953],
 [4485, 3415, 1647, 1915, 3697, 2644, 289, 2920, 3373, 3836],
 [722, 3956, 697, 4029, 2040, 3953, 3535, 3329, 1277, 3953],
 [2451, 2170, 2390, 3113, 2144],
 [1903, 3895, 4183, 4010, 3053, 1428, 269, 1881, 3772]]

In [57]:
padded_text_test= pad_sequences(onehot_text_test, padding='pre', maxlen=20)
padded_text_test[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0, 2746, 1881,
         550, 1005, 2302, 2813, 3276, 2973, 3329, 1277, 3953],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4485,
        3415, 1647, 1915, 3697, 2644,  289, 2920, 3373, 3836],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  722,
        3956,  697, 4029, 2040, 3953, 3535, 3329, 1277, 3953],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 2451, 2170, 2390, 3113, 2144],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        1903, 3895, 4183, 4010, 3053, 1428,  269, 1881, 3772]],
      dtype=int32)

In [58]:
y_pred_final=model.predict(padded_text_test)
y_pred_final[:5]

143/143 [==============================] - 1s 5ms/step


array([[2.6299074e-07],
       [4.3613167e-04],
       [6.4243161e-04],
       [9.9997044e-01],
       [1.0582202e-03]], dtype=float32)

In [59]:
y_pred_final=[round(pred[0]) for pred in y_pred_final]
y_pred_final[:5]

[0, 0, 0, 1, 0]

In [63]:
df_final=df_test.drop(['author','text'],axis=1)

In [64]:
df_final

,id,title
0,20800,"Specter of Trump Loosens Tongues, if Not Purse..."
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ..."
4,20804,Keiser Report: Meme Wars (E995)
6,20806,Pelosi Calls for FBI Investigation to Find Out...
...,...,...
5194,25994,Trump on If ’Tapes’ Exist of Comey Conversatio...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...
5197,25997,"California Today: What, Exactly, Is in Your Su..."


In [65]:
df_final['label_pred']=y_pred_final

In [66]:
df_final.head()

,id,title,label_pred
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",0
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",0
4,20804,Keiser Report: Meme Wars (E995),1
6,20806,Pelosi Calls for FBI Investigation to Find Out...,0


In [67]:
df_final.to_csv('submission_bidirectional.csv', sep=',', index=False)